# Explanation of the problem and why it is interesting

# Method to solve it

# Import of libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/adrien/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-2019.07           |           py37_0          13 KB
    certifi-2019.6.16          |           py37_0         150 KB
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The following packages will be UPDATED:

  anaconda    

In [3]:
CLIENT_ID = 'DCPMWEGCH0C2PZR13QN0IB0KEJSDLZMIPARV415TX5Y52FQ1' # your Foursquare ID
CLIENT_SECRET = 'W3BY4ARSZPO4YLRLD4BHCEWHJKLZW5UJYHEHEVDPKVXGRZXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DCPMWEGCH0C2PZR13QN0IB0KEJSDLZMIPARV415TX5Y52FQ1
CLIENT_SECRET:W3BY4ARSZPO4YLRLD4BHCEWHJKLZW5UJYHEHEVDPKVXGRZXX


In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# 1 - Explore Amman

In [13]:
address = 'Amman'

geolocator = Nominatim(user_agent="my access")
location = geolocator.geocode(address)
amman_latitude = location.latitude
amman_longitude = location.longitude
print('The geograpical coordinate of Amman are {}, {}.'.format(amman_latitude, amman_longitude))

The geograpical coordinate of Amman are 31.9515694, 35.9239625.


In [ ]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius in meters

# create URL

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    amman_latitude, 
    amman_longitude, 
    radius, 
    LIMIT)

# 2 - Explore NYC

In [14]:
address = 'New York City'

geolocator = Nominatim(user_agent="n")
location = geolocator.geocode(address)
nyc_latitude = location.latitude
nyc_longitude = location.longitude
print('The geograpical coordinate of NYC are {}, {}.'.format(nyc_latitude, nyc_longitude))

The geograpical coordinate of NYC are 40.7127281, -74.0060152.


In [ ]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius in meters

# create URL

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nyc_latitude, 
    nyc_longitude, 
    radius, 
    LIMIT)
nyc_results = requests.get(url).json()

In [ ]:
venues = nyc_results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

# 3 - Explore Israel

In [15]:
address = 'Israel'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
israel_latitude = location.latitude
israel_longitude = location.longitude
print('The geograpical coordinate of Israel are {}, {}.'.format(israel_latitude, israel_longitude))

The geograpical coordinate of Israel are 30.8760272, 35.0015196.


In [16]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius in meters

# create URL

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    israel_latitude, 
    israel_longitude, 
    radius, 
    LIMIT)